In [5]:
import modulos as md
lista=['XVGUSDT']

lista=md.lista_de_monedas()

for symbol in lista:
    try:
        data = md.obtiene_historial(symbol,'30m')
        data = md.estrategia_bb(data)
        resultado = md.backtesting(data,tp_flag=True,plot_flag=False)
        if resultado[6]>1.5:
            print(f"{symbol} - retorno: {md.truncate(resultado[6],2)}% - trades: {resultado[17]}")
    except Exception as falla:
        pass  

EOSUSDT - retorno: 1.66% - trades: 7
XLMUSDT - retorno: 2.29% - trades: 5
XTZUSDT - retorno: 6.2% - trades: 7
BATUSDT - retorno: 1.89% - trades: 7
THETAUSDT - retorno: 8.94% - trades: 10
ZRXUSDT - retorno: 2.11% - trades: 5


c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:313: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)



Error: APIError(code=-1121): Invalid symbol. - line: 236 - file: modulos.py - par: DEFIUSDT



c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:313: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)


RUNEUSDT - retorno: 4.21% - trades: 6
SUSHIUSDT - retorno: 4.32% - trades: 8
SRMUSDT - retorno: 2.25% - trades: 5
STORJUSDT - retorno: 11.73% - trades: 4
FTMUSDT - retorno: 3.9% - trades: 7
ENJUSDT - retorno: 5.65% - trades: 5
CTKUSDT - retorno: 4.2% - trades: 6
ALICEUSDT - retorno: 4.21% - trades: 10

Error: APIError(code=-1121): Invalid symbol. - line: 236 - file: modulos.py - par: 1000SHIBUSDT


Error: APIError(code=-1121): Invalid symbol. - line: 236 - file: modulos.py - par: BTCDOMUSDT


Error: APIError(code=-1121): Invalid symbol. - line: 236 - file: modulos.py - par: 1000XECUSDT

CELOUSDT - retorno: 4.91% - trades: 7
CTSIUSDT - retorno: 2.09% - trades: 8

Error: APIError(code=-1121): Invalid symbol. - line: 236 - file: modulos.py - par: FOOTBALLUSDT


Error: APIError(code=-1121): Invalid symbol. - line: 236 - file: modulos.py - par: 1000LUNCUSDT


Error: APIError(code=-1121): Invalid symbol. - line: 236 - file: modulos.py - par: LUNA2USDT


Error: APIError(code=-1121): Invalid s